<a href="https://colab.research.google.com/github/jackie-Gung/Colab_ESAA/blob/main/2022_04_15_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 필요한 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [18]:
# 데이터 전처리 함수
from sklearn.preprocessing import LabelEncoder

# NULL 처리 함수
def fillna(df):
  df['Age'].fillna(df['Age'].mean(), inplace=True)
  df['Cabin'].fillna('N', inplace=True)
  df['Embarked'].fillna('N', inplace=True)
  df['Fare'].fillna(0, inplace=True)
  return df

# 머신러닝 알고리즘에 불필요한 속성 제거 
def drop_features(df):
  df.drop(['PassengerId', 'Name','Ticket'],axis=1,inplace=True)
  return df

# 레이블 인코딩 수행
def format_features(df):
  df['Cabin'] = df['Cabin'].str[:1]
  features = ['Cabin','Sex','Embarked']
  for feature in features:
    le = LabelEncoder()
    le = le.fit(df[feature])
    df[feature] = le.transform(df[feature])
  return df

# 앞서 설정한 data preprocessing 함수 호출
def transform_features(df):
  df = fillna(df)
  df = drop_features(df)
  df = format_features(df)
  return df

## **Chap.3 평가**
### **개요**
---
    - 머신러닝은 데이터 가공/변환 -> 모델 학습/예측 -> 평가 프로세스로 구성됨
    - 머신러닝 모델은 여러 가지 방법으로 예측 성능을 평가할 수 있음
    - **성능 평가 지표(Evaluation Metric)**
      - 일반적으로 모델이 **회귀냐 분류** 냐에 따라 여러 종류로 나뉨
      - 회귀는 대부분 실제값과 예측값의 오차 평균값에 기반함 (오차 절대값의 평균 등 예측 오차를 가지고 정규화 수준을 재가공하는 방법)
      - 분류도 회귀처럼 실제와 예측 데이터의 차이에 기반하지만, 단순히 이러한 정확도만 가지고 판단하면 잘못된 평가 결과에 빠질 수 있음
    - 이번 장에서는 **분류에 사용되는 성능 평가 지표**에 대해 알아볼 것
    - 특히 0과 1로 결정값이 한정되는 **이진 분류의 성능 평가 지표**에 대해 배울 것 (더 중요시되는 경우가 많기 때문)
    - 분류 성능 평가 지표
      - 정확도(Accuracy)
      - 오차행렬(Confusion Matrix)
      - 정밀도(Precision)
      - 재현율(Recall)
      - F1 스코어
      - ROC AUC
    - 분류는 결정 클래스 값 종류의 유형에 따라 **긍정/부정과 같은 2개의 결과값만**을 가지는 **이진 분류**와, **여러 개의 결정 클래스 값**을 가지는 **멀티 분류**로 나뉨

### **1. 정확도(Accuracy)**
#### **1) 정의: 실제 데이터에서 예측 데이터가 얼마나 같은지를 판단하는 지표**
$$정확도(Accuarcy) = \frac {예측 결과가 동일한 데이터 건수}{전체 예측 데이터 건수}$$

#### **2) 정확도 특징**
- 정확도는 직관적으로 모델 예측 성능을 나타내는 평가 지표
- 이진 분류의 경우, 데이터의 구성에 따라 ML 모델의 성능을 왜곡할 수 있기 때문에, 정확도 수치 하나만 가지고 성능을 평가하지 않음
---


- 이전의 타이타닉 예제에서 탑승객이 남자인 경우보다 여자일 때 생존 확률이 높았기 때문에 별다른 알고리즘 적용 없이 성별로만 평가한다면, 여자인 경우 생존, 남자인 경우에는 사망으로 예측해도 전과 비슷한 수치가 나올 수 있음
- 따라서 사이킷런의 BaseEstimator 클래스를 상속받아 학습을 하지 않고, 성별에 따라 생존자를 예측하는 단순한 Classifier를 생성할 것
- 사이킷런은 BaseEstimator를 상속받으면 Customized 형태의 Estimator를 개발자가 생성 가능


In [22]:
from sklearn.base import BaseEstimator

class MyDummyClassifier(BaseEstimator):
  # fit() 메서드는 아무것도 학습하지 않음
  def fit(self, X, y=None):
    pass
  # predict() 메서드는 Sex feature 1이면 0, 0이면 1로 예측
  def predict(self, X):
    pred = np.zeros((X.shape[0],1))
    for i in range(X.shape[0]):
      if X['Sex'].iloc[i] == 1:
        pred[i] = 0
      else:
        pred[i] = 1
    return pred
  

In [9]:
# 타이타닉 데이터 불러오기
from google.colab import drive
drive.mount('/content/drive')
titanic = pd.read_csv('/content/drive/MyDrive/titanic_train.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# feature, target 데이터셋 분리
y_titanic = titanic['Survived']
X_titanic = titanic.drop('Survived', axis = 1)

# feature 데이터셋 전처리
X_titanic = transform_features(X_titanic)

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_titanic, y_titanic,
                                                    test_size=0.2, random_state=0)


In [25]:
# 위에서 생성한 Dummy Classifier를 이용해 학습/예측/평가 수행

myclf = MyDummyClassifier()
myclf.fit(X_train, y_train)

mypredictions = myclf.predict(X_test)
print('Dummy Classifier의 정확도는: {0:.4f}'.format(accuracy_score(y_test, mypredictions)))

Dummy Classifier의 정확도는: 0.7877


- 이렇게 단순한 알고리즘으로 예측해도 78.77%로 꽤 높은 수치가 나올 수 있기 때문에 정확도를 평가 지표로 사용할 때는 신중해야 함
- 특히 불균형한 레이블 값 분포에서 ML 모델의 성능을 판단할 경우에 정확도는 적합한 평가 지표가 아님
---

- MNIST 데이터 세트 변환하여 불균형한 데이터 세트로 만든 뒤 정확도 지표 적용 시 어떤 문제가 발생하는지 볼 것
- MNIST: 0~9까지 숫자 이미지 픽셀 정보를 가지며, 멀티 레이블 분류를 위한 데이터 세트

In [26]:
# 필요한 라이브러리와 불균형한 데이터 세트와 Dummy Classfier 생성하기
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.metrics import accuracy_score

class MyFakeClassifier(BaseEstimator):
  def fit(self,X,y):
    pass
  
  # 입력값으로 들어오는 X 데이터 세트의 크기만큼 모두 0값으로 만들어서 반환함
  def predict(self,X):
    return np.zeros((len(X),1),dtype=bool)

# sklearn의 내장 데이터 세트인 load_digits()를 이용해 MNIST 데이터 로딩하기
digits = load_digits()

# digits 번호가 7이면 True, 이를 astype(int)로 1로 변환
# 7이 아니면 False이고 0으로 변환
y = (digits.target == 7).astype(int)
X_train,X_test,y_train,y_test = train_test_split(digits.data, y, random_state=11)


In [28]:
# 불균형한 레이블 데이터 분포도 확인
print('레이블 테스트 세트 크기: ', y_test.shape)
print('테스트 세트 레이블 0과 1의 분포도')
print(pd.Series(y_test).value_counts())

# Dummy Classifier로 학습/예측/정확도 평가
fakeclf = MyFakeClassifier()
fakeclf.fit(X_train,y_test)

fakepred = fakeclf.predict(X_test)
print('모든 예측을 0으로 해도 정확도는: {:.3f}'.format(accuracy_score(y_test,fakepred)))

레이블 테스트 세트 크기:  (450,)
테스트 세트 레이블 0과 1의 분포도
0    405
1     45
dtype: int64
모든 예측을 0으로 해도 정확도는: 0.900


- 결론
      - 단순히 predict()의 결과를 np.zeros()로 모두 0값으로 반환했음에도 불구하고 예측 정확도는 90%임 -> 말도 안되는 결과
      - 정확도가 가지는 분류 평가 지표로서의 한계를 극복하기 위해 다른 분류 지표와 함께 적용해야 함

### **2. 오차 행렬**
#### **1) 정의:** 학습된 분류 모델이 예측을 수행하면서 **얼마나 헷갈리고 있는지**도 함께 보여주는 지표 (이진 분류에서 성능 지표로 잘 활용됨)
#### **2) 특징**
- 4분면 행렬에서 실제와 예측 레이블 클래스 값이 어떤 유형을 가지고 매핑되는지 나타냄
- 4분면의 왼쪽, 오른쪽을 예측된 클래스 값 기준으로 negative, positive로 분류함
- 4분면의 위,아래를 실제 클래스 값을 기준으로 negativem positive로 분류함

#### **3) TN,FP,FN,TP**
- TN: True Negative (0,0)
- FP: False Positive (0,1)
- FN: False Negative (1,0)
- TP: True Positive (1,1)

#### **4) confusion matrix**

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, fakepred)

array([[405,   0],
       [ 45,   0]])

- 결과
      - 출력된 오차 행렬: ndarray
      - TN: array[0,0] = 405
      - FN: array[1,0] = 45
      - FP: array[0,1] = 0
      - TP: array[1,1] = 0

$$정확도 = \frac {예측 결과가 동일한 데이터 건수}{전체 예측 데이터 건수} = \frac {(TN + TP)}{(TN+FP+FN+TP)}$$

- 보통 많은 데이터들은 중점적으로 찾아야하는 매우 적은 수의 결과값에 positive를 설정함
- 그런데 불균형한 이진 분류 데이터셋에서는 positive 데이터 건수가 너무 적어 이러한 데이터에 기반한 ML 알고리즘은 주로 positive보단 negative로 예측하려는 경향이 강해짐
- 결과적으로 TN은 커지고, 그 외 나머지는 작아짐

### **3. 정밀도와 재현율**
#### **1) 정의:** POSITIVE 데이터 세트의 예측 성능에 좀 더 초점을 맞춘 평가 지표
$$정밀도 = \frac {TP}{(FP+TP)}$$
$$재현율 = \frac {TP}{(FN+TP)}$$

#### **2) 정밀도**
- 예측을 positive로 한 대상 중에 예측과 실제 값이 positive로 일치한 데이터의 비율
- positive 예측 성능을 더욱 정밀하게 측정하기 위한 평가 지표로, 양성 예측도로도 불림

#### **3) 재현율**
- 실제 값이 positive인 대상 중에 예측과 실제 값이 positive로 일치한 데이터의 비율
- 민감도(Sensitivy) or TPR(True Positive Rate)

- 정밀도와 재현율은 이진 분류 모델의 업무 특성에 따라 특정 평가 지표가 더 중요한 지표로 간주됨
- **재현율이 중요한 경우**
  - 실제 POSITIVE 양성 데이터를 NEGATIVE로 잘못 판단하여 업무상 큰 영향이 발생하는 경우
  - 예: 암 판단 모델, 금융 사기 적발 모델
  - FN을 낮추는데 초점

- **정밀도가 중요한 경우**
  - 실제 NEGATIVE 음성 데이터를 POSITIVE로 잘못 판단하여 업무상 큰 영향이 발생하는 경우
  - 예: 스팸메일 여부 판단 모델
  - FP를 낮추는데 초점
---

- 정밀도 계산: **precision_score()**
- 재현율 계싼: **recall_score()**
- 평가를 간편하게 적용하기 위해 **get_clf_eval()**
  - confusion_matrix
  - accuracy
  - precision
  - recall

In [33]:
# get_clf_eval() 함수
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

def get_clf_eval(y_test, pred):
  confusion = confusion_matrix(y_test,pred)
  accuracy = accuracy_score(y_test,pred)
  precision = precision_score(y_test,pred)
  recall = recall_score(y_test,pred)
  print('오차 행렬')
  print(confusion)
  print('정확도: {0:.4f}, 정밀도: {1:,4f}, 재현율: {2:.4f}'.format(accuracy,precision,recall))

In [32]:
# 원본 데이터 재로딩
titanic = pd.read_csv('/content/drive/MyDrive/titanic_train.csv')

# logistic 회귀 기반 생존자 예측
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# feature, target 데이터셋 분리
y_titanic = titanic['Survived']
X_titanic = titanic.drop('Survived', axis = 1)

# feature 데이터셋 전처리
X_titanic = transform_features(X_titanic)

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X_titanic, y_titanic,
                                                    test_size=0.2, random_state=11)

lr_clt = LogisticRegression()

lr_clt.fit(X_train,y_train)
pred = lr_clt.predict(X_test)
get_clf_eval(y_test, pred) # 값은 나오는데 왜 value error가 생기지..? 그리고 값도 책과 다름

오차 행렬
[[104  14]
 [ 13  48]]


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


ValueError: ignored

#### 4) 정밀도/재현율 트레이드오프
- 분류하려는 업무의 특성상, 정밀도 또는 재현율이 특별히 강조돼야 할 경우 분류의 결정 임계값을 조정해 수치를 높일 수 있음
- 하지만 정밀도와 재현율은 상호 보완적 평가 지표이기에 강제로 어느 한쪽을 높이면, 다른 수치는 떨어질 확률이 높음 ->이를 Trade-Off
- 사이킷런의 분류 알고리즘은 예측 데이터가 특정 레이블에 속하는지 계산하기 위해 먼저 개별 레이블별로 결정 확률을 구해야함
- 이후에 예측 확류이 큰 레이블값으로 예측하게 됨